In [11]:
import numpy as np
import copy
import functools
import operator
import random

In [17]:
import torch
import matplotlib.pyplot as plt

torch.manual_seed(42)

In [12]:
class Tropical:
    def __init__(self,e,n=1):
        self.e = e
        self.n = n
        
    def __add__(self,other):
        if self.e == other.e:
            n1 = self.n+other.n
        elif self.e > other.e:
            n1 = other.n
        elif self.e< other.e:
            n1 = self.n
        return Tropical(min(self.e,other.e),n1)
    
    def __mul__(self,other):
        return Tropical(self.e+other.e,self.n*other.n)
    
    def __str__(self):
        return "Tropical(%d,%d)" %(self.e,self.n)
    
    def __truediv__(self,other):
        return Tropical(self.e-other.e,self.n-other.n)

In [13]:
def oinsum(eq, *arrays):
    
    """A ``einsum`` implementation for ``numpy`` object arrays.
    """
    lhs, output = eq.split('->')
    inputs = lhs.split(',')

    sizes = {}
    for term, array in zip(inputs, arrays):
        for k, d in zip(term, array.shape):
            sizes[k] = d

    out_size = tuple(sizes[k] for k in output)
    out = np.empty(out_size, dtype=object) # malloc

    inner = [k for k in sizes if k not in output]
    inner_size = [sizes[k] for k in inner]

    for coo_o in np.ndindex(*out_size):

        coord = dict(zip(output, coo_o))

        def gen_inner_sum():
            for coo_i in np.ndindex(*inner_size):
                coord.update(dict(zip(inner, coo_i)))

                locs = [tuple(coord[k] for k in term) for term in inputs]
                elements = (array[loc] for array, loc in zip(arrays, locs))
                yield functools.reduce(operator.mul, elements)

        out[coo_o] = functools.reduce(operator.add, gen_inner_sum())

    return out

In [14]:
def Eigen_Tropical(B):
    '''
    Return the max eigen value of a given tropical matrix (numpy array) B
    '''
    e = 10**(-5) # Error
    N = 5*10**(1) # Total run time
    dim = B.shape[0]
    phi = np.array([Tropical(random.random(),1) for _ in range(dim)])
    
    flag = True
    count = 0
    lamb = np.sum(phi)
    while (flag == True) and (count<=N):
        print("phi = [%f,%f]"%(phi[0].e,phi[1].e))
    
        phi1 = B@phi
        lamb1 = np.sum(phi1)
        phi1 = phi1/lamb1
        error = abs((lamb1.e-lamb.e)/(lamb.e))
        if error<= e:
            flag = False
        else:
            phi = phi1
            lamb = lamb1
        print("%d time: Error = %f, lambda = %f"%(count,error,lamb1.e))
        count += 1
    print('lambda = %f' % (lamb1.e))
    return lamb1.e

In [15]:
d = 2 # fixed
D = 20 # PEPS size 

B = np.array([[Tropical(1, 1), Tropical(-1, 1)],
        [Tropical(-1, 1), Tropical(1, 1)]], dtype=object)
G = oinsum('ij,jk,kl,lm->ijkl',B,B,B,B)
G = G.reshape(d,d**2,d)

TODO: Now it's to try to build a function to translate a normal function to a tropical one

In [16]:
S = np.vectorize(lambda x: Tropical(x)) # Vectorize would work for Tropical

> Now we could try use the `loss` function as a black box to do the optimize work 

In [17]:
def loss(A):
    '''
    obtain a loss from a given matrix A (numpy.ndarray)
    '''
    Asymm = symmetrize(A)
    print("already symmetrized")
    TA = S(Asymm)
    print("done")
    T1 = oinsum('xa,xby,yc->abc',TA,G,TA)
    return T1

In [18]:
def symmetrize(A):
    Asymm = A.reshape(d,D,D)
    Asymm = (Asymm + np.transpose(Asymm,(0, 2, 1)))/2
    return Asymm.reshape(d,D**2)

In [19]:
if __name__ == '__main__':
    TestA = np.ones((d,D,D))
    %timeit -n3 loss(TestA)

already symmetrized
done
already symmetrized
done
already symmetrized
done
already symmetrized
done
already symmetrized
done
already symmetrized
done
already symmetrized
done
already symmetrized
done
already symmetrized
done
already symmetrized
done
already symmetrized
done
already symmetrized
done
already symmetrized
done
already symmetrized
done
already symmetrized
done
already symmetrized
done
already symmetrized
done
already symmetrized
done
already symmetrized
done
already symmetrized
done
57.8 s ± 787 ms per loop (mean ± std. dev. of 7 runs, 3 loops each)
